In [5]:
import os
import numpy as np
import argparse
# from ouster import client, pcap
from ouster.sdk import client, pcap
from contextlib import closing
from more_itertools import nth
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import seaborn as sns
from scipy.stats import skew, kurtosis
from tqdm import tqdm

In [6]:
folder_path =  r"D:\LiDAR\RawData"

In [7]:
folders = ["18_09_2024_2gSodium","20_09_2024_2gSodium","24_10_2024_2gSodium"]
degen_start_time = np.array([127, 140, 125]) * 10
folder_list = [folder_path + "\\" + i for i in folders]
folder_list

['D:\\LiDAR\\RawData\\18_09_2024_2gSodium',
 'D:\\LiDAR\\RawData\\20_09_2024_2gSodium',
 'D:\\LiDAR\\RawData\\24_10_2024_2gSodium']

# Extract data from Full format experiments

In [13]:
def StatisticalCalc(frame):
    params = []
    for ind in range(frame.shape[-1]):
        data = frame[:,:,ind].flatten()
        meanval = np.mean(data)
        varval = np.std(data)
        medval = np.median(data)
        q1val = np.percentile(data, 25)
        q3val = np.percentile(data, 75)
        minval = np.min(data)
        maxval = np.max(data)
        ptsval = np.count_nonzero(data)
        skewval = skew(data)
        kurtval = kurtosis(data)
        params.extend([meanval, varval, minval, q1val, medval, q3val, maxval, skewval, kurtval, ptsval])
    return params

In [11]:
folders = folder_list
fps = 10
df = pd.DataFrame(index = np.arange(30000), 
                  columns=['Exp', 'Folder', 'Mode', 'Frame', 
                           'MeanR', 'VarR', 'MinR', 'Q1R','MedR', 'Q3R','MaxR', 'SkewR', 'KurtR','PtsR',
                           'MeanSig', 'VarSig','MinSig', 'Q1Sig','MedSig', 'Q3Sig','MaxSig','SkewSig', 'KurtSig','PtsSig', 
                           'MeanRef', 'VarRef', 'MinRef','Q1Ref', 'MedRef', 'Q3Ref','MaxRef','SkewRef', 'KurtRef','PtsRef',
                           'MeanIR', 'VarIR', 'MinIR','Q1IR','MedIR', 'Q3IR','MaxIR', 'SkewIR', 'KurtIR','PtsIR'])
row_counter = 0
folder_count = -1
for folder in folders:
    folder_count += 1
    rootfolder = folder.split("\\")[-1]
    for subfolder in os.listdir(folder):
        full_path = folder + "\\" + subfolder
        if "Full" not in subfolder:
            continue
        counter = 0
        if os.path.isdir(full_path):
            file_name = 'OS-1-32-122147000654-2048x10'
            intdata = np.empty(shape=(32,2048, 4, 1))
            fulldata = np.empty(shape=(32,2048, 4, 1))
            frame_counter = 0
            if os.path.isfile(full_path + "\\" + file_name + '.json') :
                metadata_path = full_path + "\\" + file_name + '.json'
                pcap_path = full_path + "\\" + file_name + '.pcap'
                print(f"File {metadata_path} Located...")
                with open(metadata_path, 'r') as f:                    
                    metadata = client.SensorInfo(f.read())
                    print(f"Loading {pcap_path} File...")
                    source = pcap.Pcap(pcap_path, metadata)
                    with closing(client.Scans(source)) as scans:
                        for scan in tqdm(scans):
                            rangedata = client.destagger(metadata, scan.field(client.ChanField.RANGE))
                            sigdata = client.destagger(metadata, scan.field(client.ChanField.SIGNAL)) 
                            refdata = client.destagger(metadata, scan.field(client.ChanField.REFLECTIVITY))
                            irdata = client.destagger(metadata, scan.field(client.ChanField.NEAR_IR))
                            testdata = np.stack([rangedata, sigdata, refdata, irdata], axis=-1)
                            if counter >= degen_start_time[folder_count]:                                
                                if frame_counter < 10:                             
                                    intdata = np.concatenate([intdata, np.expand_dims(testdata, axis=-1)], axis=3) 
                                    frame_counter += 1
                                else:
                                    frame_counter = 0
                                    MeanFrame = np.mean(intdata[:,:,:,1:], axis=3)
                                    header = [rootfolder, subfolder, 'Degen', counter]
                                    statparams = StatisticalCalc(MeanFrame)                                
                                    df.iloc[row_counter] = header + statparams
                                    intdata = np.empty(shape=(32,2048, 4, 1))
                                    row_counter += 1
                                    counter += 1
                            else:
                                MeanFrame = testdata
                                header = [rootfolder, subfolder, 'Gen', counter]
                                statparams = StatisticalCalc(MeanFrame)                                
                                df.loc[row_counter] = header + statparams                                
                                row_counter += 1
                                counter += 1    
df.dropna().to_csv("ConsolidatedData.csv")                  

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2404\4085788877.py:31: DeprecationWarning: pcap.Pcap(...) is deprecated: Use pcap.PcapMultiPacketReader(...).single_source(0) instead. This API is planned to be removed in Q4 2024.
  source = pcap.Pcap(pcap_path, metadata)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2404\4085788877.py:32: DeprecationWarning: client.Scans(...) is deprecated: Use client.ScansMulti(...).single_source(0) or the appropriate scan source directly instead. This API is planned to be removed in Q4 2024.
  with closing(client.Scans(source)) as scans:


File D:\LiDAR\RawData\18_09_2024_2gSodium\01_Full\OS-1-32-122147000654-2048x10.json Located...
Loading D:\LiDAR\RawData\18_09_2024_2gSodium\01_Full\OS-1-32-122147000654-2048x10.pcap File...
loading metadata from []


9310it [05:20, 29.03it/s]


In [12]:
df.head()

,Exp,Folder,Mode,Frame,MeanR,VarR,MinR,Q1R,MedR,Q3R,...,MeanIR,VarIR,MinIR,Q1IR,MedIR,Q3IR,MaxIR,SkewIR,KurtIR,PtsIR
0,18_09_2024_2gSodium,01_Full,Gen,0,0.0,0.0,0,0.0,0.0,0.0,...,58.726562,51457.316895,0,0.0,0.0,0.0,1431,3.70717,12.14755,4224
1,18_09_2024_2gSodium,01_Full,Gen,1,18.607635,280659.075023,0,0.0,0.0,0.0,...,186.031067,132355.322643,0,0.0,0.0,0.0,2079,1.669564,1.452365,14560
2,18_09_2024_2gSodium,01_Full,Gen,2,16.623215,12983.759171,0,0.0,0.0,0.0,...,186.067932,132382.361291,0,0.0,0.0,0.0,2048,1.669992,1.459481,14560
3,18_09_2024_2gSodium,01_Full,Gen,3,16.804977,13085.453711,0,0.0,0.0,0.0,...,186.073975,132402.059042,0,0.0,0.0,0.0,2063,1.66974,1.455439,14560
4,18_09_2024_2gSodium,01_Full,Gen,4,16.549622,12914.719187,0,0.0,0.0,0.0,...,185.962265,132253.163752,0,0.0,0.0,0.0,2038,1.670121,1.457063,14560


In [83]:
df = pd.DataFrame(index = np.arange(30000), columns=['Exp', 'Folder', 'Frame', 
                                                     'MeanR', 'VarR', 'MinR', 'Q1R','MedR', 'Q3R','MaxR', 'SkewR', 'KurtR','PtsR',
                                                     'MeanSig', 'VarSig','MinSig', 'Q1Sig','MedSig', 'Q3Sig','MaxSig','SkewSig', 'KurtSig','PtsSig', 
                                                     'MeanRef', 'VarRef', 'MinRef','Q1Ref', 'MedRef', 'Q3Ref','MaxRef','SkewRef', 'KurtRef','PtsRef',
                                                     'MeanIR', 'VarIR', 'MinIR','Q1IR','MedIR', 'Q3IR','MaxIR', 'SkewIR', 'KurtIR','PtsIR'
                                                     'Target', 'Smoke', 
                                                     'Sig_40Deg_Mean', 'Sig_40Deg_Var', 'Sig_20Deg_Mean', 'Sig_20Deg_Var'])

# counter = 0
# TargetCnt = 0
# idx = 0
# rangedata = []
# temp_arr = []
# sigdata = []
# refdata = []
# irdata = []
# ind_min = 829
# ind_max = 1285
# #26_02_2024
# ind_min = 845
# ind_max = 1299 
# n = (ind_max - ind_min) // 10
# n2 = (1122 - 973) // 3
# update = 0
# SkipFolders = [] # ['00_BG', '00_BG1', '00_Sodium_Generation']
# frame_counter = -1
for i in range(len(folder_list)): 
    print(i)
    update = 0
    file_name = 'OS-1-32-122147000654-2048x10'
#     file_name = 'OS-1-32-122146001278-2048x10' # 26_02_2024
    if os.path.isfile(folder_path + '/' + folder_list[i] + '/' + file_name + '.json') :
        metadata_path = folder_path + '/' + folder_list[i] + '/' + file_name + '.json'
        pcap_path = folder_path + '/' + folder_list[i] + '/' + file_name + '.pcap'
        print("Executed")
        with open(metadata_path, 'r') as f:
            metadata = client.SensorInfo(f.read())
        source = pcap.Pcap(pcap_path, metadata)
        FrameCnt = 0
        TargetCnt = TargetCnt + 1
        with closing(client.Scans(source)) as scans:
            print(folder_list[i])
            for scan in scans:
                frame_counter += 1
                if frame_counter % 10 == 0:
                    #print(frame_counter)
                    pass
                else:
                    continue                
                ref_range1 = client.destagger(metadata, scan.field(client.ChanField.RANGE))
                ref_range = (ref_range1[ref_range1 != 0]).flatten()
                ref_sig1 = client.destagger(metadata, scan.field(client.ChanField.SIGNAL)) 
#                 if update == 0:
#                     x, y = np.where(ref_sig1)
#                     ind_max = y.max()
#                     ind_min = y.min()
#                     n = (ind_max - ind_min) // 10
#                     update = 1
#                     print('Min: ', ind_min, 'Max: ', ind_max, 'N: ', n)
                temp_arr = [calcAvg_cust(ref_sig1[:, k:k+n].flatten()) for k in range(ind_min, ind_max, n)]
                #temp_arr = [calcVar_cust(ref_sig1[:, k:k+n].flatten()) for k in range(ind_min, ind_max, n)]
                sigdata.append(temp_arr)
                #ref_sig = (ref_sig1[ref_sig1 != 0]).flatten()  
                ref_sig = ref_sig1.flatten()
                ref_sig = ref_sig[ref_sig != 0]
                ref_reflect = client.destagger(metadata, scan.field(client.ChanField.REFLECTIVITY))
                ref_reflect = (ref_reflect[ref_reflect != 0]).flatten()
                ref_ir = client.destagger(metadata, scan.field(client.ChanField.NEAR_IR))
                temp_arr = [calcAvg_cust(ref_ir[:, k:k+n].flatten()) for k in range(ind_min, ind_max, n)]
                #temp_arr = [calcVar_cust(ref_ir[:, k:k+n].flatten()) for k in range(ind_min, ind_max, n)]
                irdata.append(temp_arr)
                ref_ir = (ref_ir[ref_ir != 0]).flatten()
                ref_range1 = client.destagger(metadata, scan.field(client.ChanField.RANGE))
                temp_arr = [calcAvg_cust(ref_range1[:, k:k+n2].flatten()) for k in range(973, 1122, n2)]
                #temp_arr = [calcVar_cust(ref_range1[:, k:k+n2].flatten()) for k in range(973, 1122, n2)]
                rangedata.append(temp_arr)
                ref_range = (ref_range1[ref_range1 != 0]).flatten()
                ref_sig = client.destagger(metadata, scan.field(client.ChanField.SIGNAL))
                ref_sig = (ref_sig[ref_sig != 0]).flatten()
                ref_reflect = client.destagger(metadata, scan.field(client.ChanField.REFLECTIVITY))
                temp_arr = [calcAvg_cust(ref_reflect[:, k:k+n].flatten()) for k in range(973, 1122, n)]
                #temp_arr = [calcVar_cust(ref_reflect[:, k:k+n].flatten()) for k in range(973, 1122, n)]
                refdata.append(temp_arr)
                ref_reflect = (ref_reflect[ref_reflect != 0]).flatten()
                ref_ir = client.destagger(metadata, scan.field(client.ChanField.NEAR_IR))
                ref_ir = (ref_ir[ref_ir != 0]).flatten()
                df['Sig_40Deg_Mean'].iloc[counter], df['Sig_40Deg_Var'].iloc[counter] = calcAvgVar_cust(ref_sig1[:, ind_min+(n*3):ind_min+(n*7)].flatten())
                df['Sig_20Deg_Mean'].iloc[counter], df['Sig_20Deg_Var'].iloc[counter] = calcAvgVar_cust(ref_sig1[:, ind_min+(n*4):ind_min+(n*6)].flatten())
                
                df['MeanSigStrength'].iloc[counter], df['Points'].iloc[counter], df['VarSigStrength'].iloc[counter] = calcAvg(ref_sig)
                df['MeanRange'].iloc[counter], df['PointsRange'].iloc[counter], df['VarRange'].iloc[counter] = calcAvg(client.destagger(metadata, scan.field(client.ChanField.RANGE)))
                df['MeanReflectivity'].iloc[counter], df['PointsReflect'].iloc[counter], df['VarReflectivity'].iloc[counter]  = calcAvg(ref_reflect)
                df['MeanIR'].iloc[counter], df['PointsIR'].iloc[counter], df['VarIR'].iloc[counter] = calcAvg(ref_ir)
                df['Folder'].iloc[counter] = str(folder_list[i])
                df['Frame'].iloc[counter] = FrameCnt
                df['Target'].iloc[counter] = i+1 #TargetCnt

                df['Q1Sig'].iloc[counter] = np.quantile(ref_sig, 0.25) if len(ref_sig) != 0 else 0
                df['MedSig'].iloc[counter] = np.quantile(ref_sig, 0.5) if len(ref_sig) != 0 else 0
                df['Q3Sig'].iloc[counter] = np.quantile(ref_sig, 0.75) if len(ref_sig) != 0 else 0
                df['MinSig'].iloc[counter] = ref_sig.min() if len(ref_sig) != 0 else 0
                df['MaxSig'].iloc[counter] = ref_sig.max() if len(ref_sig) != 0 else 0
                    
                          
                df['Q1Range'].iloc[counter] = np.quantile(ref_range, 0.25) if len(ref_range) != 0 else 0
                df['MedRange'].iloc[counter] = np.quantile(ref_range, 0.5) if len(ref_range) != 0 else 0
                df['Q3Range'].iloc[counter] = np.quantile(ref_range, 0.75) if len(ref_range) != 0 else 0
                df['MinRange'].iloc[counter] = ref_range.min() if len(ref_range) != 0 else 0
                df['MaxRange'].iloc[counter] = ref_range.max() if len(ref_range) != 0 else 0
                          
                df['Q1Reflect'].iloc[counter] = np.quantile(ref_reflect, 0.25) if len(ref_reflect) != 0 else 0
                df['MedReflect'].iloc[counter] = np.quantile(ref_reflect, 0.5) if len(ref_reflect) != 0 else 0
                df['Q3Reflect'].iloc[counter] = np.quantile(ref_reflect, 0.75) if len(ref_reflect) != 0 else 0
                df['MinReflect'].iloc[counter] = ref_reflect.min() if len(ref_reflect) != 0 else 0
                df['MaxReflect'].iloc[counter] = ref_reflect.max() if len(ref_reflect) != 0 else 0
                
                df['Q1IR'].iloc[counter] = np.quantile(ref_ir, 0.25) if len(ref_ir) != 0 else 0
                df['MedIR'].iloc[counter] = np.quantile(ref_ir, 0.5) if len(ref_ir) != 0 else 0
                df['Q3IR'].iloc[counter] = np.quantile(ref_ir, 0.75) if len(ref_ir) != 0 else 0
                df['MinIR'].iloc[counter] = ref_ir.min() if len(ref_ir) != 0 else 0
                df['MaxIR'].iloc[counter] = ref_ir.max() if len(ref_ir) != 0 else 0
                
                
                if idx < len(Smoke):
                    df['Smoke'].iloc[counter] = Smoke[int(idx)]
                else:
                    df['Smoke'].iloc[counter] = np.nan
                counter = counter + 1
                FrameCnt = FrameCnt + 1
                #if FrameCnt >= 1000 and folder_list[i] not in SkipFolders:
                    #!= '00_Sodium Generation' and folder_list[i] != '23_04_02_Sample1' and 
                    #folder_list[i] != '24_04_05_VesselOpen':
                #    break
                if FrameCnt % 100 == 1:
                    print(FrameCnt, end=" ")
            if counter > 0: #To remove last frame from each file
                counter = counter -1

        idx = idx + 1
            

In [9]:
['2', '3'].extend(a)

In [12]:
a = [1,2]
b=[3,4]
a.extend(b)
a


[1, 2, 3, 4]

In [71]:
x = np.concatenate([x, np.expand_dims(testdata, axis=-1)], axis=3)
x.shape

(32, 2048, 4, 2)

In [61]:
intdata=np.expand_dims(testdata, axis=-1)
intdata.shape

(32, 2048, 4, 1)

In [69]:
np.mean(x, axis=3).shape

(32, 2048, 4)

In [72]:
x[:, :, :, 0]

array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       ...,

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0.